# Intro to Machine learning, Final Project

In this project, we are given the Enron dataset, and the goal is to identify the persons of interest in the Enron scandal. We are provided with financial features, like salary, stock options, etc... and some email features, detailing the exchanges between the collaborators. We first import the datasets.

## Importation of the data:

## Data cleaning and outliers

We start with some data cleaning, identifying some outliers and absurdities: